## ArcGIS Notes:

### Feature Class Basics: 
* [Link](https://desktop.arcgis.com/en/arcmap/10.3/manage-data/geodatabases/feature-class-basics.htm)

### Reading Geometries: 
* [Link](http://resources.arcgis.com/en/help/main/10.1/index.html#//002z0000001t000000)

In [ ]:
import os
import math

import numpy as np
import pandas as pd

import arcpy
import arcgis

import utils

%load_ext autoreload
%autoreload 2

In [ ]:
config = utils.load_config("config.yml")

In [ ]:
polygons = {}
for island in config["islands"]:
    path = utils.path_to_shoreline(island)
    pair = max(arcpy.da.TableToNumPyArray(path, ["OID@", "SHAPE@AREA"]), key=lambda p: p[1])
    poly = arcpy.da.SearchCursor(path, ["SHAPE@"], where_clause=f"FID = {pair[0]}").next()[0]
    polygons[island] = poly

In [ ]:
rectangles = []

size = config["pix_res"] * config["data_extraction"]["pix_dim"] + config["data_extraction"]["pix_pad"]
step = int(size * (1 - config["data_extraction"]["overlap"]))

for island, polygon in polygons.items():
    
    rectangles.append([])
    
    polyline = polygon.boundary()
    polyline_perimeter = int(polyline.getLength("PLANAR", "METERS"))
    print("Perimeter: {0:.2f}".format(polyline_perimeter))

    spatial_reference = polygon.spatialReference
    point_A = arcpy.PointGeometry(polyline.firstPoint).projectAs(spatial_reference)
    
    for perimeter_position in range(0, polyline_perimeter, step):
        
        if len(rectangles[-1]) % 100 == 0:
            N, percentage = len(rectangles[-1]), perimeter_position / polyline_perimeter
            print("{0} rectangles collected, {1:.2f}% complete.".format(N, percentage))
        
        multipoint = point_A.buffer(size).boundary().intersect(polyline, 1)
                            
        point_B, _ = sorted(
            map(lambda pt: (pt, polyline.measureOnLine(pt)), multipoint), 
            key=lambda x: (x[1] < perimeter_position, x[1] - perimeter_position)
        )[0]

        point_B = arcpy.PointGeometry(point_B).projectAs(spatial_reference)
        
        assert math.isclose(point_A.distanceTo(point_B), size, rel_tol=10) # high rel_tol for high variance estimate
        
        line_AB = arcpy.Polyline(arcpy.Array([point_A.firstPoint, point_B.firstPoint])).projectAs(spatial_reference)
        
        point_C = utils.isoscelese_vertex_point(line_AB, size * 2.0)
        
        triangle = arcpy.Polyline(arcpy.Array([point_A.firstPoint, point_B.firstPoint, point_C.firstPoint]))
                
        rectangles[-1].append(triangle.hullRectangle)
        
        point_A = polyline.positionAlongLine(perimeter_position)
        

In [ ]:
shorelines = [[] for _ in range(len(polygons))]
for i, polygon in enumerate(polygons):
    for section in polygon:
        for point in section:
            if point:
                shorelines[i].append([float(point.X), float(point.Y)])
            else:
                raise ValueError("Please ensure all shoreline Polygons have had internal polygons removed. +\
                                  See \'remove_inner_polygon\' in data/utils for more info.")
    shorelines[i] = np.array(shorelines[i])

In [ ]:
for i, shoreline in enumerate(shorelines):
    dist = np.concatenate([np.linalg.norm(shoreline[:-1] - shoreline[1:], axis=1), [0]])[:, None]
    stats = pd.DataFrame(np.concatenate([
        shoreline,
        dist,
        dist / 4
    ], axis=1), columns=["X", "Y", "dist (m)", "dist (px)"])
    print(f"stats for {islands[i]}")
    display(stats.describe())
